In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn import preprocessing
from sklearn import cross_validation
from sklearn.feature_selection import VarianceThreshold
with open('./max/estims.json') as data_file: 
    estims_data = json.load(data_file)[1]['clinics']
print len(estims_data)
#with open('./max/меланома_все.json') as data_file: #2
#with open('./max/рак_груди_все.json') as data_file: #0
#with open('./max/рак_простаты_все.json') as data_file: #4
with open('./max/рак_шейки_матки_все.json') as data_file: #1
#with open('./max/рак_щитовидки_все.json') as data_file: #3
    cancer_data = json.load(data_file)[0]['clinics']
print len(cancer_data)

In [ ]:
FEATURES = ["views", "large", "top", "price_level", "speed_level", "count_orders", "count_orders_success", "count_reviews", "count_reviews_for_illness", "top_proc_cli", "gen_proc_cli", "top_oper_cli", "gen_oper_cli", "doc_count", "doctor_prof_count", "doctor_doc_count", "max_exp_doc", "mean_exp_doc", "max_h_index", "mean_h_index", "doc_top_diag", "doc_gen_diag", "doc_top_oper", "doc_gen_oper", "ill_cli", "lang_cli", "specialist", "experience", "variety", "price"]

In [ ]:
clinic_ids = [i['id'] for i in estims_data]
#estims_data.pop(clinic_ids.index('0'))
print sorted(clinic_ids)
print sorted([i['id'] for i in cancer_data])

In [ ]:
def extract_data(clinic, clinic_estim):
    all_proc = clinic['procedures']['diagnostics'] + clinic['procedures']['operations']
    all_doc = clinic['doctors']
    X = []
    X+=[int(clinic['views'])]
    X+=[int(clinic['large'])]
    X+=[int(clinic['top'])]
    X+=[int(clinic['price_level'])]
    X+=[int(clinic['speed_level'])]
    X+=[int(clinic['count_orders'])]
    X+=[int(clinic['count_orders_success'])]
    X+=[int(clinic['count_reviews'])]
    X+=[int(clinic['count_reviews_for_illness'])]
    X+=[sum([int(i['top']) for i in all_proc])] #top_proc_cli
    X+=[len(all_proc)] #gen_proc_cli
    X+=[sum([int(i['top']) for i in clinic['procedures']['operations']])] #top_oper_cli
    X+=[len(clinic['procedures']['operations'])] #gen_oper_cli
    X+=[len(clinic['doctors'])] #doc_count
    X+=[[i['degree'] for i in all_doc].count(u'Профессор')]    #doctor_prof_count
    X+=[[i['degree'] for i in all_doc].count(u'Доцент')]    #doctor_doc_count
    try:
        X+=[max([int(i['experience']) for i in all_doc])] #max_exp_doc
        X+=[np.mean([int(i['experience']) for i in all_doc])] #mean_exp_doc
    except ValueError:
        X+=[0,0]
    try:
        X+=[max([int(i['h_index']) for i in all_doc])] #max_h_index
        X+=[np.mean([int(i['h_index']) for i in all_doc])] #mean_h_index
    except ValueError:
        X+=[0,0]
    #doc_top_diag
    tops = [i['procedures']['diagnostics'] for i in all_doc]
    ids_diad = [i['id'] for i in sum(tops,[]) if i['top']=='1']
    X+=[len(list(set(ids_diad)))]
    #doc_gen_diag
    tops = [i['procedures']['diagnostics'] for i in all_doc]
    ids = [i['id'] for i in sum(tops,[])]
    X+=[len(list(set(ids)))]
    #doc_top_oper
    tops = [i['procedures']['operations'] for i in all_doc]
    ids = [i['id'] for i in sum(tops,[]) if i['top']=='1']
    X+=[len(list(set(ids)))]
    #doc_gen_oper
    tops = [i['procedures']['operations'] for i in all_doc]
    ids = [i['id'] for i in sum(tops,[])]
    X+=[len(list(set(ids)))]
    #ill_cli
    tops = [i['illnesses'] for i in all_doc]
    ids = [i['id'] for i in sum(tops,[])]
    X+=[len(list(set(ids)))]
    #lang_cli
    tops = [i['languages'] for i in all_doc]
    ids = [i['id'] for i in sum(tops,[])]
    X+=[len(list(set(ids)))]
    X+=[int(clinic_estim['specialist'])]
    X+=[int(clinic_estim['experience'])]
    X+=[int(clinic_estim['variety'])]
    X+=[int(clinic_estim['price'])]
    y=int(clinic_estim['rating'])
    return [X, y]

In [ ]:
def get_X_y_from(t_data):
    clinic_ids = [i['id'] for i in t_data]
    t_cleaned_data = [extract_data(cancer_data[clinic_ids.index(i['id'])], i) for i in estims_data]
    X = np.array([i[0] for i in t_cleaned_data])
    y = np.array([i[1] for i in t_cleaned_data])
    return X,y

In [ ]:
def process_with(X,y, info=False, short=False, return_short = False, new_coef = [], ts=0.2):
    train_X, test_X, train_y, test_y = cross_validation.train_test_split(X, y, test_size = ts, random_state = 3)
    regr = linear_model.LinearRegression(fit_intercept=False, normalize=False)
    if new_coef != []:
        print "new coefs"
        regr.coef_ = new_coef
    regr.fit(train_X, train_y)
    if info:
        print "Total: %d, train: %d, test: %d" %(len(X), len(train_X), len(test_X))
        print("Residual sum of squares: %.2f"% np.mean((regr.predict(test_X) - test_y) ** 2))
        print("Train absolute: %.2f"% np.mean(abs(regr.predict(train_X) - train_y)))
        print("Test absolute: %.2f"% np.mean(abs(regr.predict(test_X) - test_y)))
        print("Absolute to mean: %.2f%%"% (np.mean(abs(regr.predict(test_X) - test_y))/np.mean(test_y)*100))
        print('Train variance score: %.2f' % regr.score(train_X, train_y))
        print('Test variance score: %.2f' % regr.score(test_X, test_y))
    if short:
        print np.mean(abs(regr.predict(test_X) - test_y)),regr.score(test_X, test_y)
    if return_short:
        return np.mean(abs(regr.predict(test_X) - test_y)),regr.score(test_X, test_y)
    return regr

In [ ]:
X, y = get_X_y_from(cancer_data)

In [ ]:
for i in range(1,10):
    regr = process_with(X, y, info=True, ts=i/10.0)
    print

In [ ]:
regr = process_with(X, y, info=True, ts=0.3)

In [ ]:
import warnings
warnings.filterwarnings("ignore")
for i in range(0,len(y)):
    print "Real: %f \t Predicted: %f" %(y[i], regr.predict(X[i]))

In [ ]:
print "Features sorted by their score:"
#f = sorted(zip(map(lambda x: round(x, 4), regr.coef_), FEATURES), reverse=True)
f = zip(map(lambda x: round(x, 4), regr.coef_), FEATURES)
print regr.intercept_
print len(FEATURES)
for i in f:
    #print "%4f \t %s" %(i[0], i[1])
    print i[0]

In [ ]:
ind = [map(lambda x: x[1], f).index(i) for i in FEATURES]
for i in ind:
    print i+1

In [ ]:
plt.plot([i[13] for i in X])
plt.show()

In [ ]:
print FEATURES[13]
[i[13] for i in X]

In [ ]:
for k in range(0,26):
    print FEATURES[k], 10 - [i[k] for i in X].count(0)